In [14]:
import os
import glob
import sys
import copy
import zlib
import pathlib
import tempfile
import subprocess
import itertools
import random
import json
import math
import scipy as sp
import scipy.optimize
import cma
import skimage.draw
from scipy.optimize import differential_evolution, minimize, basinhopping
from sklearn.metrics import pairwise_distances
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from skimage.segmentation import flood_fill

sys.path.append("../src")

from anno_designer import ad_to_grid, BLOCKING, object_positions, ID_TOWNHALL, ID_GUILDHOUSE
from utils import crop_to_nonzero, TOWNHALL_COVERAGE

In [15]:
input_glob = os.path.join("..", "resources", "Anno Designer - Empty Layouts", "Old World - Cape Trelawney", "**", "*_empty.ad")

In [16]:
# num_sources = 24
radius = 20
source_size = 4
e = 1e-8

In [17]:
# ensure the scikit methods return the correct townhall coverage
coverage_sklearn = skimage.draw.disk(center=(19.5, 19.5), radius=20)
coverage_actual = np.nonzero(TOWNHALL_COVERAGE)
assert (coverage_sklearn[0] == coverage_actual[0]).all()
assert (coverage_sklearn[1] == coverage_actual[1]).all()

In [18]:
def distance_from_set(s) -> np.ndarray:
    distance = np.where(s, 0.0, np.inf)
    visited = e < s # set grows outward
    for k in itertools.count(1):
        # expand to 3x3 neighbourhood
        visited = e < scipy.ndimage.uniform_filter(np.where(visited, 1.0, 0.0), 3)
        update = (distance == np.inf) & visited
        if not update.any():
            break
        distance[update] = k
    return distance

def signed_distance_from_boundary(s) -> np.ndarray:
    return distance_from_set(s) - np.maximum(0, -1 + distance_from_set(~s))

In [19]:


def vector_to_sources(grid, x, grid_mode):
    
    if grid_mode:

        phi = x[0]
        rotation = np.array([[np.cos(phi), np.sin(phi)], [-np.sin(phi), np.cos(phi)]])
        translation = x[1:3]

        sources = []
        for u in range(-10, 10):
            for v in range(-10, 10):
                coords = np.array((2 * u + (v % 2), np.sqrt(3) * v))
                coords = rotation @ coords + translation
                coords = radius * coords
                coords = coords.astype(int)
                footprint = skimage.draw.rectangle(
                    start=coords - source_size / 2,
                    extent=(source_size, source_size),
                    shape=grid.shape,
                )
                if footprint[0].size and grid[footprint].all():
                    sources.append(coords)
    else: 
        sources = x.reshape((-1, 2))
        sources = sources[0, :] + sources[1:, :]

    return np.array(sources)

    # sources_transformed = list(map(affine_transform, sources))
    # return np.array(sources_transformed)

def sources_to_grid(grid, sources):
    a = np.zeros_like(grid, dtype=int)
    for s in sources:
        d = skimage.draw.disk(center=np.round(s) - .5, radius=radius, shape=a.shape)
        a[d] += 1
    return a

def constraints(distance_grid, sources):
    bounds = np.array(distance_grid.shape) - 1

    for s in sources:

        # ensure coordinates are in bounds
        yield abs(s - np.clip(s, 0, bounds)).sum()

        # ensure building is buildable
        footprint = skimage.draw.rectangle(
            start=np.array(s) - source_size / 2,
            extent=(source_size, source_size),
            shape=distance_grid.shape,
        )
        samples = distance_grid[footprint]
        if samples.size == 0:
            samples = distance_grid
        yield max(0, samples.max(initial=0)) 

    # check lower-triangle-entries of distance matrix
    if sources.size:
        distances = pairwise_distances(np.round(sources))
        distances_tril = distances[np.tril_indices_from(distances, -1)]
        distance_constraints = np.maximum(0, 2 * radius -  distances_tril)
        distance_constraints = np.square(distance_constraints)

        # increase values in (e, 1) to 1 to prevent underconstraining
        distance_constraints = np.where(e < distance_constraints, np.maximum(1, distance_constraints), distance_constraints)

        yield distance_constraints.sum()
    else:
        yield 0

def objective_fn(grid, distance_grid, x, grid_mode, constraint_weight):
    sources = vector_to_sources(grid, x, grid_mode=grid_mode)
    coverage = sources_to_grid(grid, sources)
    reward_coverage = (grid * (coverage > 0)).sum()
    reward_coverage -= (source_size * source_size) * len(sources)
    value = reward_coverage
    value -= constraint_weight * sum(constraints(distance_grid, sources))
    return -value

def sources_to_image(grid, sources):
    output_shape = grid.shape[0], grid.shape[1], 3
    img = np.zeros(output_shape, dtype=np.uint8)
    coverage = sources_to_grid(grid, sources)
    img[:, :, 0] += (255 * (coverage == 0) * (grid == 1)).astype(np.uint8)
    img[:, :, 1] += (255 * (coverage == 0) * (grid == 1)).astype(np.uint8)
    img[:, :, 0] += (255 * (coverage > 1) * (grid == 1)).astype(np.uint8)
    img[:, :, 1] += (255 * (coverage == 1) * (grid == 1)).astype(np.uint8)
    for s in sources:
        r = skimage.draw.rectangle(
            start=np.array(s) - source_size / 2,
            extent=(source_size, source_size),
            shape=grid.shape,
        )
        img[r[0], r[1], :] = 255
    return img

In [20]:
def source_to_ad_object(i, j):
    obj = dict()
    obj["Identifier"] = "Town hall"
    obj["Label"] = ""
    obj["Position"] = f"{i-source_size//2},{j-source_size//2}"
    obj["Size"] = f"{source_size},{source_size}"
    obj["Icon"] = "A7_townhall"
    obj["Template"] = "Guildhouse"
    obj["Color"] = {"A": 255, "R": 238, "G": 130, "B": 238}
    obj["Borderless"] = True
    obj["Road"] = False
    obj["Radius"] = radius
    obj["InfluenceRange"] = -2.0
    obj["BlockedAreaLength"] = 0.0
    obj["BlockedAreaWidth"] = 0.0
    obj["Direction"] = "Up"
    return obj

In [21]:
def solve(grid, distance_grid):

    best_value = np.inf
    best_sources = None

    for seed in range(3):

        x0 = [0, 0, 0]
        sigma0 = 1.0

        options = cma.CMAOptions()
        options.set("bounds", [[-np.pi, -1, -1], [+np.pi, 1, 1]])
        options.set("seed", seed)
        options.set("popsize", 32)
        options.set("tolflatfitness", 10)

        es = cma.CMAEvolutionStrategy(x0, sigma0, options)
        while not es.stop():
            solutions = es.ask()
            values = [objective_fn(grid, distance_grid, x, grid_mode=True, constraint_weight=0) for x in solutions]
            es.tell(solutions, values)
            es.disp()
            sources = vector_to_sources(grid, es.best.x, grid_mode=True)
            yield sources
        es.result_pretty()

        bounds_lower = 2 * [-1000] + len(sources) * [0, 0]
        bounds_upper = 2 * [+1000] + len(sources) * list(grid.shape)

        x0 = np.concatenate(([0, 0], sources.flatten()))
        sigma0 = 0.1
        constraint_weight = 100

        options = cma.CMAOptions()
        options.set("bounds", [bounds_lower, bounds_upper])
        options.set("seed", seed)
        # options.set("tolstagnation", 10)
        options.set("tolfunhist", 1e-7)
        # options.set("maxiter", 1000)
        options.set("popsize", 64)
        # options.set("tolflatfitness", 32)
        # options.set("tolfun", 0)
        # options.set("tolfunhist", 0)
        options.set("integer_variables", list(range(len(x0))))

        for epoch in range(100):

            es = cma.CMAEvolutionStrategy(x0, sigma0, options)
            # for _ in range(100):
            while not es.stop():
                solutions = es.ask()
                values = [objective_fn(grid, distance_grid, x, grid_mode=False, constraint_weight=constraint_weight) for x in solutions]
                es.tell(solutions, values)
                es.disp()
                sources = vector_to_sources(grid, es.best.x, grid_mode=False)
                yield sources
            es.result_pretty()

            constraint_loss = sum(constraints(distance_grid, sources))
            if 0 == constraint_loss:
                break

            constraint_weight *= 3
            best_value = es.best.f
            x0 = es.best.x

        if es.best.f < best_value:
            best_value = es.best.f
            best_sources = sources
        else:
            print(f"{epoch=} did not yield improvement.")

    yield best_sources
        
    # x_sol = es.best.x
    # x_sol

In [22]:
input_paths = glob.glob(input_glob, recursive=True)
input_paths

['..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\community_island\\community_island_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\moderate_l_01\\moderate_l_01_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\moderate_l_02\\moderate_l_02_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\moderate_l_03\\moderate_l_03_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\moderate_l_04\\moderate_l_04_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\moderate_l_05\\moderate_l_05_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\Large Islands\\moderate_l_06\\moderate_l_06_empty.ad',
 '..\\resources\\Anno Designer - Empty Layouts\\Old World - Cape Trelawney\\L

In [ ]:
for input_path in input_paths:

    with open(input_path) as f:
        input_json = json.load(f)
    input_data = ad_to_grid(input_json)

    images = []
    distance_grid = distance_from_set(input_data)

    for iteration, sources in enumerate(solve(input_data, distance_grid)):

        img = sources_to_image(input_data, sources)
        img = np.repeat(img, 2, 0)
        img = np.repeat(img, 2, 1)
        img = np.pad(img, ((100, 0), (0, 0), (0, 0)))
        image = Image.fromarray(img)
        draw = ImageDraw.Draw(image)
        font = ImageFont.truetype("arial.ttf", 16)
        draw.text(
            (0, 0),
            f"Iteration : {iteration}",
            font=font,
        )

        coverage = sources_to_grid(input_data, sources)
        coverage_tiles = ((coverage > 0) & input_data).sum()
        draw.text(
            (0, 20),
            f"Tiles covered : {coverage_tiles}",
            font=font,
        )
        constraint_loss = sum(constraints(distance_grid, sources))
        draw.text(
            (0, 40),
            f"Constraint loss : {round(constraint_loss, 3)}",
            font=font,
        )

        images.append(image)
        image.save(os.path.join("..", "resources", "optimization", "latest.png"))

    output_json = copy.deepcopy(input_json)
    output_json["Objects"] = list(filter(
        lambda obj: obj["Identifier"] in BLOCKING,
        output_json["Objects"]))
    for i, j in sources:
        obj = source_to_ad_object(j, i)
        output_json["Objects"].append(obj)

    output_path = os.path.join("..", "resources", "optimization", "islands", f"{os.path.basename(input_path)}")
    # output_path = input_path.replace("_empty.ad", "_th_opt.ad")
    with open(output_path, "w") as f:
        json.dump(output_json, f, indent=4)

    # animation_path = os.path.join("..", "resources", "optimization", "animation.gif")
    # animation_path = output_path + ".gif"
    # images[0].save(animation_path, save_all=True, append_images=images[1:], duration=1, loop=0)

c:\Users\volke\PycharmProjects\anno-tools\.venv\Lib\site-packages\cma\evolution_strategy.py:1312: UserWarning:

Sampling standard deviation i=1 (and 1 others) at iteration 0 multiplied by 0.6666555556481476 to stds[1]=0.6666666666666666



(16_w,32)-aCMA-ES (mu_w=9.2,w_1=19%) in dimension 3 (seed=675017, Sat Oct  4 07:01:09 2025)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     32 -2.220000000000000e+04 1.0e+00 1.01e+00  6e-01  8e-01 0:00.6
    2     64 -2.264200000000000e+04 1.3e+00 9.32e-01  4e-01  7e-01 0:01.1
    3     96 -2.221700000000000e+04 1.4e+00 8.71e-01  4e-01  6e-01 0:01.7
    9    288 -2.307300000000000e+04 2.2e+00 9.68e-01  4e-01  1e+00 0:04.8
   18    576 -2.298000000000000e+04 6.1e+00 1.25e+00  4e-01  2e+00 0:09.3
   28    896 -2.318500000000000e+04 1.3e+01 1.62e+00  3e-01  2e+00 0:14.4
   40   1280 -2.216200000000000e+04 4.0e+01 2.11e+00  5e-01  2e+00 0:20.4
   54   1728 -2.203300000000000e+04 2.7e+01 3.19e+00  6e-01  2e+00 0:27.5
   70   2240 -2.244800000000000e+04 2.9e+01 2.98e+00  3e-01  9e-01 0:35.6
   89   2848 -2.202800000000000e+04 1.7e+02 4.09e+00  6e-01  2e+00 0:45.0
  100   3200 -2.180500000000000e+04 2.9e+02 2.32e+00  2e-01  5e-01 0:50.4
  121   3872 -2.179600

c:\Users\volke\PycharmProjects\anno-tools\.venv\Lib\site-packages\cma\evolution_strategy.py:1312: UserWarning:

Sampling standard deviation i=0 (and 47 others) at iteration 0 multiplied by 2.0 to stds[0]=0.2



    1     64 -2.112900000000000e+04 1.0e+00 9.55e-02  2e-01  2e-01 0:00.2
    2    128 -2.112700000000000e+04 1.1e+00 9.77e-02  2e-01  2e-01 0:00.4
    3    192 -2.138140542042759e+04 1.1e+00 1.04e-01  2e-01  2e-01 0:00.6
   20   1280 -2.178680542316158e+04 1.4e+00 5.10e-01  1e+00  1e+00 0:03.7
   42   2688 -2.235647350640715e+04 1.7e+00 3.36e-01  7e-01  8e-01 0:07.7
   69   4416 -2.278000000000000e+04 1.8e+00 1.95e-01  4e-01  5e-01 0:12.8
  100   6400 -2.299800000000000e+04 2.1e+00 1.34e-01  3e-01  4e-01 0:18.6
  139   8896 -2.314500000000000e+04 2.4e+00 9.50e-02  2e-01  3e-01 0:25.7
  183  11712 -2.325600000000000e+04 3.0e+00 7.90e-02  2e-01  3e-01 0:33.8
  200  12800 -2.325600000000000e+04 3.1e+00 4.61e-02  2e-01  2e-01 0:36.9
  225  14400 -2.325600000000000e+04 3.8e+00 2.46e-02  2e-01  2e-01 0:41.5
termination on tolfunhist=1e-07
final/bestever f-value = -2.325600e+04 -2.327900e+04 after 14400/11292 evaluations
incumbent solution: [  1.   2.  46. 163.  35. 202.  59. 234. ...]
std d

c:\Users\volke\PycharmProjects\anno-tools\.venv\Lib\site-packages\cma\evolution_strategy.py:1312: UserWarning:

Sampling standard deviation i=1 (and 1 others) at iteration 0 multiplied by 0.6666555556481476 to stds[1]=0.6666666666666666



...

Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     32 -2.213700000000000e+04 1.0e+00 8.36e-01  5e-01  7e-01 0:00.5
    2     64 -2.279100000000000e+04 1.3e+00 9.05e-01  4e-01  9e-01 0:01.0
    3     96 -2.294500000000000e+04 1.5e+00 7.85e-01  3e-01  7e-01 0:01.4
   10    320 -2.237200000000000e+04 4.4e+00 6.76e-01  1e-01  6e-01 0:04.7
   19    608 -2.232400000000000e+04 7.0e+00 5.23e-01  3e-02  3e-01 0:09.0
   30    960 -2.309200000000000e+04 3.3e+00 6.11e-01  5e-02  3e-01 0:14.2
   43   1376 -2.291900000000000e+04 8.2e+00 6.81e-01  4e-02  3e-01 0:20.4
   58   1856 -2.427900000000000e+04 4.4e+00 1.80e+00  1e-01  2e-01 0:27.6
   75   2400 -2.443000000000000e+04 2.3e+01 9.30e-01  2e-03  2e-02 0:35.9
   94   3008 -2.446200000000000e+04 3.8e+01 9.07e-01  7e-04  5e-03 0:45.3
  100   3200 -2.447900000000000e+04 4.4e+01 1.04e+00  3e-04  2e-03 0:48.2
  108   3456 -2.447900000000000e+04 3.9e+01 6.59e-01  6e-05  5e-04 0:52.2
termination on tolfun=1e-11
termination 

c:\Users\volke\PycharmProjects\anno-tools\.venv\Lib\site-packages\cma\evolution_strategy.py:1312: UserWarning:

Sampling standard deviation i=0 (and 49 others) at iteration 0 multiplied by 2.0 to stds[0]=0.2



Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     64 -2.227000000000000e+04 1.0e+00 9.68e-02  2e-01  2e-01 0:00.2
    2    128 -2.230780542316158e+04 1.1e+00 9.97e-02  2e-01  2e-01 0:00.4
    3    192 -2.234400000000000e+04 1.1e+00 1.08e-01  2e-01  2e-01 0:00.6
   19   1216 -2.268300000000000e+04 1.4e+00 2.92e-01  6e-01  7e-01 0:03.7
   40   2560 -2.326900000000000e+04 1.8e+00 2.56e-01  5e-01  6e-01 0:07.7
   67   4288 -2.370900000000000e+04 2.0e+00 1.89e-01  4e-01  5e-01 0:12.9
   99   6336 -2.384900000000000e+04 2.2e+00 1.32e-01  3e-01  4e-01 0:19.0
  100   6400 -2.385000000000000e+04 2.2e+00 1.30e-01  3e-01  4e-01 0:19.2
  142   9088 -2.408400000000000e+04 2.6e+00 9.46e-02  2e-01  3e-01 0:27.4
  189  12096 -2.415500000000000e+04 3.2e+00 7.58e-02  2e-01  4e-01 0:36.5
  200  12800 -2.415500000000000e+04 3.4e+00 5.50e-02  2e-01  3e-01 0:38.6
  216  13824 -2.415500000000000e+04 3.5e+00 3.08e-02  2e-01  2e-01 0:41.7
termination on tolfunhist=1e-07
final/be

c:\Users\volke\PycharmProjects\anno-tools\.venv\Lib\site-packages\cma\evolution_strategy.py:1312: UserWarning:

Sampling standard deviation i=1 (and 1 others) at iteration 0 multiplied by 0.6666555556481476 to stds[1]=0.6666666666666666



Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     32 -2.416000000000000e+04 1.0e+00 1.05e+00  5e-01  1e+00 0:00.5
    2     64 -2.548700000000000e+04 1.6e+00 9.38e-01  4e-01  8e-01 0:01.0
    3     96 -2.461100000000000e+04 1.7e+00 8.36e-01  4e-01  7e-01 0:01.5
   10    320 -2.478900000000000e+04 3.1e+00 8.55e-01  4e-01  7e-01 0:05.0
   19    608 -2.447100000000000e+04 1.5e+00 1.18e+00  3e-01  6e-01 0:09.3
   30    960 -2.546700000000000e+04 2.6e+00 1.27e+00  2e-01  7e-01 0:14.7
   43   1376 -2.458700000000000e+04 3.6e+00 1.70e+00  2e-01  9e-01 0:21.1
   58   1856 -2.474900000000000e+04 2.7e+01 3.42e+00  2e-01  2e+00 0:28.5
   74   2368 -2.467500000000000e+04 2.3e+01 3.79e+00  2e-01  6e-01 0:36.8
